# Play with first models

In [87]:
import pandas as pd
#get data:
file_features = 'dengue_features_train.csv'
file_labels = 'dengue_labels_train.csv'

df_features = pd.read_csv("../data/"+file_features)
df_labels = pd.read_csv("../data/"+file_labels)

#split data initially for train-validation and TEST:
from sklearn.model_selection import train_test_split
X = df_features
y = df_labels
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Begin transformation pipeline:

In [85]:
#fix date, drop extras
X_train.loc[:, 'week_start_date'] = pd.to_datetime(X_train.loc[:, 'week_start_date'], yearfirst=True)
X_train.drop(['year', 'weekofyear'], axis=1, inplace=True)
y_train.drop(['city', 'year', 'weekofyear'], axis=1, inplace=True)

/home/ant_on_su/miniconda3/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/home/ant_on_su/miniconda3/lib/python3.7/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [86]:
#fill NANs
X_train.fillna(method='ffill', inplace=True)

/home/ant_on_su/miniconda3/lib/python3.7/site-packages/pandas/core/frame.py:3790: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


In [9]:
#One-hot ecode 'city'
X_train = pd.get_dummies(X_train)

In [10]:
X_train.head()

,week_start_date,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,reanalysis_avg_temp_k,reanalysis_dew_point_temp_k,reanalysis_max_air_temp_k,...,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm,city_iq,city_sj
1417,2009-10-01,0.206300,0.255257,0.279957,0.206657,57.18,298.034286,299.571429,296.344286,307.4,...,57.18,17.947143,8.514286,27.950000,12.225000,35.2,20.5,33.1,1,0
446,1998-11-26,0.050100,0.065750,0.166243,0.146757,0.00,299.232857,299.321429,295.492857,301.4,...,0.00,16.867143,2.014286,26.557143,5.685714,30.0,22.8,29.8,0,1
936,2000-07-01,0.192886,0.132257,0.340886,0.247200,25.41,296.740000,298.450000,295.184286,307.3,...,25.41,16.651429,8.928571,26.400000,10.775000,32.5,20.7,3.0,1,0
1032,2002-05-07,0.227257,0.178243,0.177386,0.210114,101.30,297.355714,298.435714,296.444286,304.4,...,101.30,18.024286,8.042857,27.500000,11.525000,34.0,21.3,25.4,1,0
495,1999-11-05,0.062750,0.103400,0.173457,0.165883,4.84,299.960000,299.942857,296.531429,302.0,...,4.84,18.014286,1.885714,27.442857,6.542857,32.2,23.9,40.9,0,1


In [11]:
X_train.dtypes

week_start_date                          datetime64[ns]
ndvi_ne                                         float64
ndvi_nw                                         float64
ndvi_se                                         float64
ndvi_sw                                         float64
precipitation_amt_mm                            float64
reanalysis_air_temp_k                           float64
reanalysis_avg_temp_k                           float64
reanalysis_dew_point_temp_k                     float64
reanalysis_max_air_temp_k                       float64
reanalysis_min_air_temp_k                       float64
reanalysis_precip_amt_kg_per_m2                 float64
reanalysis_relative_humidity_percent            float64
reanalysis_sat_precip_amt_mm                    float64
reanalysis_specific_humidity_g_per_kg           float64
reanalysis_tdtr_k                               float64
station_avg_temp_c                              float64
station_diur_temp_rng_c                         

# model1: GBReg

In [88]:
#TODO: find more elegant way
X_test.drop(['week_start_date'], axis=1, inplace=True)
X_train.drop(['week_start_date'], axis=1, inplace=True)
X_train.city = X_train.city.map({'sj':0, 'iq':1})
X_test.city = X_test.city.map({'sj':0, 'iq':1})

/home/ant_on_su/miniconda3/lib/python3.7/site-packages/pandas/core/generic.py:4405: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [89]:
X_test.fillna(value=0, inplace=True)
X_train.fillna(value=0, inplace=True)

In [90]:
y_test = y_test.iloc[:,-1]
y_train = y_train.iloc[:,-1]

In [115]:
from sklearn import ensemble
from sklearn.metrics import mean_absolute_error
#Fit regression model
params = {'n_estimators': 500, 'max_depth': 8, 'min_samples_split': 2,
          'learning_rate': 0.01, 'loss': 'ls'}
clf = ensemble.GradientBoostingRegressor(**params)

clf.fit(X_train, y_train)
mse = mean_absolute_error(y_test, clf.predict(X_test).astype('int'))
print("MAE: %.4f" % mse)

MAE: 11.4829


## Make Submittion

In [143]:
df_TEST = pd.read_csv("../data/dengue_features_test.csv")
df_TEST.drop(['week_start_date'], axis=1, inplace=True)
df_TEST.fillna(value=0, inplace=True)
df_TEST.city = df_TEST.city.map({'sj':0, 'iq':1})
PRED = clf.predict(df_TEST).astype('int')

In [144]:
df_PRED = df_TEST.iloc[:,:3]
df_PRED.city = df_PRED.city.map({0:'sj', 1:'iq'})
df_PRED['total_cases'] = pd.Series(PRED)

In [145]:
df_PRED.head()

,city,year,weekofyear,total_cases
0,sj,2008,18,4
1,sj,2008,19,7
2,sj,2008,20,4
3,sj,2008,21,6
4,sj,2008,22,13


In [147]:
df_PRED.to_csv("../output/PRED_gbreg.csv", index=False)